In [1]:
from pprint import pprint
import math
import pandas as pd
from transformers import AutoModelForMaskedLM
from transformers import AutoTokenizer
from transformers import TrainingArguments
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer
from datasets import Dataset

In [2]:
df = pd.read_excel("./data/telesale/100_calls.xlsx", engine="openpyxl", sheet_name="100 calls")

In [3]:
df.head()

,fileName,start,end,speaker,sentiment,text,Intent,Unnamed: 7,Unnamed: 8
0,VUQC83EOO969H66VRLO5RMEPC8089Q00_2021-07-16_11...,0.81,1.27,0.0,3.0,a lô,NaN,NaN,NaN
1,NaN,1.86,2.76,0.0,3.0,anh đơn hả anh,Agent_VerifyCustomerName,NaN,NaN
2,NaN,4.62,7.86,0.0,3.0,ờ anh em là hương nè em gọi đến cho anh từ bên...,Agent_Self Introduction,Agent_CompanyIntroduction,NaN
3,NaN,8.04,34.32,0.0,3.0,a lô cho em hỏi xíu ha là vợ anh chị lép á còn...,Agent_VerifyCustomerName,NaN,NaN
4,NaN,34.68,35.07,0.0,3.0,anh anh,NaN,NaN,NaN


In [4]:
# Just printing
texts = df["text"].tolist()
pprint(texts[:10])
print(len(texts))

['a lô',
 'anh đơn hả anh',
 'ờ anh em là hương nè em gọi đến cho anh từ bên ngân hàng vi bi banh á',
 'a lô cho em hỏi xíu ha là vợ anh chị lép á còn dùng số điện thoại cũ không à '
 'em hỏi là vợ anh mà chị lép á anh còn dùng số điện thoại cũ không anh ha số '
 'bốn tám bốn đuôi á đổi số rồi hả ủa ba không hai anh ơi anh đọc giùm em được '
 'không à vậy ạ',
 'anh anh',
 'ừm',
 'anh đưa điện thoại giùm em',
 nan,
 'à chị lép hả chị',
 'à hiện tại hiện thì trong cái đợt dịch đợt dịch này nè em à cũng bên em cũng '
 'có đang hỗ trợ lại cho khách hàng cũ á']
9368


In [5]:
""" Remove nan """
texts = [text for text in texts if isinstance(text, str)]
pprint(texts[:10])
print(len(texts))

['a lô',
 'anh đơn hả anh',
 'ờ anh em là hương nè em gọi đến cho anh từ bên ngân hàng vi bi banh á',
 'a lô cho em hỏi xíu ha là vợ anh chị lép á còn dùng số điện thoại cũ không à '
 'em hỏi là vợ anh mà chị lép á anh còn dùng số điện thoại cũ không anh ha số '
 'bốn tám bốn đuôi á đổi số rồi hả ủa ba không hai anh ơi anh đọc giùm em được '
 'không à vậy ạ',
 'anh anh',
 'ừm',
 'anh đưa điện thoại giùm em',
 'à chị lép hả chị',
 'à hiện tại hiện thì trong cái đợt dịch đợt dịch này nè em à cũng bên em cũng '
 'có đang hỗ trợ lại cho khách hàng cũ á',
 'cái điện số điện thoại cũ của chị thì em gọi nó mấy nay không có được á nên '
 'là xin phép em nói sơ qua chút xíu ha']
8614


In [6]:
texts = list(set(texts))
pprint(texts[:10])
print(len(texts))

['dạ à',
 'cho nên em cứ cân nhắc đi rồi có gì tuần sau chị gọi lại cho em',
 'thứ hai là nó có hợp đồng đàng hoàng cho chị mà chứ đâu phải nói chuyện '
 'không không ví dụ à em nói em cho chị vay như vậy không đâu đâu có',
 'thì tiền mặt luôn cái ốp bồ mới ra nó cứ nó cứ điện thoại gì cũng mới ra '
 'cũng đây mấy tháng á hai ba tháng nè',
 'ờ rồi',
 'trời ơi tâm thần của mình không ăn uống gì được nữa trời ơi',
 'ừ mà họ không duyệt nữa là thôi',
 'khi mà mình có uy tín của ngân hàng rồi thì mình mới vay được ờ',
 'thì có anh có đóng tốn hai lăm ngàn không',
 'hồ sơ của chị chị được duyệt á em đăng ký cho mình thì chị chỉ cần cầm giấy '
 'chứng minh ra ngân hàng']
6519


In [7]:
dataset = Dataset.from_dict({"text": texts})
dataset

Dataset({
    features: ['text'],
    num_rows: 6519
})

In [8]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

def tokenize(samples):
    return tokenizer(samples["text"], return_special_tokens_mask=True)

tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=["text"])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/7 [00:00<?, ?ba/s]

In [9]:
RANDOM_SEED = 42
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.2, seed=RANDOM_SEED)
trainset = tokenized_dataset["train"]
evalset = tokenized_dataset["test"]

In [10]:
trainset

Dataset({
    features: ['input_ids', 'token_type_ids', 'special_tokens_mask', 'attention_mask'],
    num_rows: 5215
})

In [11]:
evalset

Dataset({
    features: ['input_ids', 'token_type_ids', 'special_tokens_mask', 'attention_mask'],
    num_rows: 1304
})

In [12]:
model = AutoModelForMaskedLM.from_pretrained("vinai/phobert-base")
model.config

RobertaConfig {
  "_name_or_path": "vinai/phobert-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 258,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "PhobertTokenizer",
  "transformers_version": "4.12.5",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 64001
}

In [13]:
training_args = TrainingArguments(
    num_train_epochs=60,
    learning_rate=1e-4,
    lr_scheduler_type="linear",
    weight_decay=0.01,
    warmup_ratio=0.06,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    logging_steps=100,
    logging_strategy="steps",
    save_steps=100,
    save_strategy="steps",
    eval_steps=100,
    evaluation_strategy="steps",
    save_total_limit=10,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    logging_dir="./logs/tapt/telesale",
    output_dir="./results/tapt/telesale",
    report_to="wandb"
)

In [14]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm_probability=0.15
)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=trainset,
    eval_dataset=evalset,
    data_collator=data_collator
)

In [16]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: special_tokens_mask.
***** Running training *****
  Num examples = 5215
  Num Epochs = 60
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 9780
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: hosjiu (use `wandb login --relogin` to force relogin)


Step,Training Loss,Validation Loss
100,4.399200,3.611763
200,3.466000,3.174999
300,3.113100,2.914603
400,2.863300,2.885626
500,2.867100,2.785375
600,2.697400,2.752808
700,2.640800,2.614743
800,2.551700,2.494638
900,2.467900,2.625203
1000,2.485600,2.539428


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: special_tokens_mask.
***** Running Evaluation *****
  Num examples = 1304
  Batch size = 16
Saving model checkpoint to ./results/tapt/telesale/checkpoint-100
Configuration saved in ./results/tapt/telesale/checkpoint-100/config.json
Model weights saved in ./results/tapt/telesale/checkpoint-100/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: special_tokens_mask.
***** Running Evaluation *****
  Num examples = 1304
  Batch size = 16
Saving model checkpoint to ./results/tapt/telesale/checkpoint-200
Configuration saved in ./results/tapt/telesale/checkpoint-200/config.json
Model weights saved in ./results/tapt/telesale/checkpoint-200/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForMaskedLM.

Deleting older checkpoint [results/tapt/telesale/checkpoint-700] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: special_tokens_mask.
***** Running Evaluation *****
  Num examples = 1304
  Batch size = 16
Saving model checkpoint to ./results/tapt/telesale/checkpoint-1800
Configuration saved in ./results/tapt/telesale/checkpoint-1800/config.json
Model weights saved in ./results/tapt/telesale/checkpoint-1800/pytorch_model.bin
Deleting older checkpoint [results/tapt/telesale/checkpoint-800] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: special_tokens_mask.
***** Running Evaluation *****
  Num examples = 1304
  Batch size = 16
Saving model checkpoint to ./results/tapt/telesale/checkpoint-1900
Configuration saved in ./results/tapt/telesale/checkpoint-1900/con

The following columns in the evaluation set  don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: special_tokens_mask.
***** Running Evaluation *****
  Num examples = 1304
  Batch size = 16
Saving model checkpoint to ./results/tapt/telesale/checkpoint-3300
Configuration saved in ./results/tapt/telesale/checkpoint-3300/config.json
Model weights saved in ./results/tapt/telesale/checkpoint-3300/pytorch_model.bin
Deleting older checkpoint [results/tapt/telesale/checkpoint-2300] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: special_tokens_mask.
***** Running Evaluation *****
  Num examples = 1304
  Batch size = 16
Saving model checkpoint to ./results/tapt/telesale/checkpoint-3400
Configuration saved in ./results/tapt/telesale/checkpoint-3400/config.json
Model weights saved in ./results/tapt/telesale/checkpoint-3400/pytorch_model.bin
Del

***** Running Evaluation *****
  Num examples = 1304
  Batch size = 16
Saving model checkpoint to ./results/tapt/telesale/checkpoint-4800
Configuration saved in ./results/tapt/telesale/checkpoint-4800/config.json
Model weights saved in ./results/tapt/telesale/checkpoint-4800/pytorch_model.bin
Deleting older checkpoint [results/tapt/telesale/checkpoint-3800] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: special_tokens_mask.
***** Running Evaluation *****
  Num examples = 1304
  Batch size = 16
Saving model checkpoint to ./results/tapt/telesale/checkpoint-4900
Configuration saved in ./results/tapt/telesale/checkpoint-4900/config.json
Model weights saved in ./results/tapt/telesale/checkpoint-4900/pytorch_model.bin
Deleting older checkpoint [results/tapt/telesale/checkpoint-3900] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresp

  Num examples = 1304
  Batch size = 16
Saving model checkpoint to ./results/tapt/telesale/checkpoint-6300
Configuration saved in ./results/tapt/telesale/checkpoint-6300/config.json
Model weights saved in ./results/tapt/telesale/checkpoint-6300/pytorch_model.bin
Deleting older checkpoint [results/tapt/telesale/checkpoint-5400] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: special_tokens_mask.
***** Running Evaluation *****
  Num examples = 1304
  Batch size = 16
Saving model checkpoint to ./results/tapt/telesale/checkpoint-6400
Configuration saved in ./results/tapt/telesale/checkpoint-6400/config.json
Model weights saved in ./results/tapt/telesale/checkpoint-6400/pytorch_model.bin
Deleting older checkpoint [results/tapt/telesale/checkpoint-5500] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForM

  Batch size = 16
Saving model checkpoint to ./results/tapt/telesale/checkpoint-7800
Configuration saved in ./results/tapt/telesale/checkpoint-7800/config.json
Model weights saved in ./results/tapt/telesale/checkpoint-7800/pytorch_model.bin
Deleting older checkpoint [results/tapt/telesale/checkpoint-6900] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: special_tokens_mask.
***** Running Evaluation *****
  Num examples = 1304
  Batch size = 16
Saving model checkpoint to ./results/tapt/telesale/checkpoint-7900
Configuration saved in ./results/tapt/telesale/checkpoint-7900/config.json
Model weights saved in ./results/tapt/telesale/checkpoint-7900/pytorch_model.bin
Deleting older checkpoint [results/tapt/telesale/checkpoint-7000] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForMaskedLM.forward` and h

Saving model checkpoint to ./results/tapt/telesale/checkpoint-9300
Configuration saved in ./results/tapt/telesale/checkpoint-9300/config.json
Model weights saved in ./results/tapt/telesale/checkpoint-9300/pytorch_model.bin
Deleting older checkpoint [results/tapt/telesale/checkpoint-8400] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: special_tokens_mask.
***** Running Evaluation *****
  Num examples = 1304
  Batch size = 16
Saving model checkpoint to ./results/tapt/telesale/checkpoint-9400
Configuration saved in ./results/tapt/telesale/checkpoint-9400/config.json
Model weights saved in ./results/tapt/telesale/checkpoint-9400/pytorch_model.bin
Deleting older checkpoint [results/tapt/telesale/checkpoint-8500] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: 

TrainOutput(global_step=9780, training_loss=1.3886506632549387, metrics={'train_runtime': 1358.795, 'train_samples_per_second': 230.278, 'train_steps_per_second': 7.198, 'total_flos': 7806330031926318.0, 'train_loss': 1.3886506632549387, 'epoch': 60.0})